<a href="https://colab.research.google.com/github/grahamstelzer/fundamentals/blob/main/transformer_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
# setup
import torch
import torch.nn as nn
import math

In [36]:
torch.manual_seed(0)
print("pytorch version: ", torch.__version__)

pytorch version:  2.9.0+cu126


In [37]:
# "config"
batch_size = 2
seq_len = 4
input_dim = 16
d_model = 8
num_heads = 2
d_ff = 32

In [38]:
# "base" mha
class MultiHeadAttention(nn.Module):

    # constructor
    def __init__(self, d_model, num_heads):
        super().__init__() # call nn.Module constructor
        assert d_model % num_heads == 0

        # member variables
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads

        # set transformations for weights
        #   q, k, v and output
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    # forward
    def forward(self, x):

        # shape usually (batch_size, seq_length, d_model) but sometimes not
        B, S, _ = x.shape

        # apply transformations
        Q = self.W_q(x)
        K = self.W_k(x)
        V = self.W_v(x)

        # change tensors from (B, S, d_model) -> (B, S, head_idx, d_model // num_heads)
        #   or in other words, split the d_model dimension into 1 section per head
        #   (this is why we need to do assert d_model % num heads == 0, otherwise uneven split)

        def reshape(t):
            t_reshaped = t.view(B, S, self.num_heads, self.head_dim)
            t_heads = t_reshaped.transpose(1, 2)
            return t_heads

        Qh = reshape(Q)
        Kh = reshape(K)
        Vh = reshape(V)


        # scaled dot product attention:
        scores = Qh @ Kh.transpose(-2, -1) # Q * K_t
        scores = scores / math.sqrt(self.head_dim) # divide by sqrt d_model accounting for multihead

        attn = scores.softmax(dim=-1) # softmax

        out = attn @ Vh # multiply by V

        out = out.transpose(1,2).contiguous().view(B, S, self.d_model) # combine heads

        out = self.W_o(out) # final linear transform

        return out




In [39]:
# feed forward (position-wise)
class FFN(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )

    def forward(self, x):
        out = self.net(x)
        return out


In [40]:
# "transformer block" - though i think this could also be referred to as "encoderblock"
#   apparently standard convention these days is to use a transformer block with toggleable
#   cross attention and attention mask, then use for both "encoder" and "decoder

class TransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        self.norm1 = nn.LayerNorm(d_model)
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.norm2 = nn.LayerNorm(d_model)
        self.ffn = FFN(d_model, d_ff)

    def forward(self, x):

        # mha and add/norm
        x_norm = self.norm1(x)
        attn_out = self.mha(x_norm)
        x = x + attn_out

        # ff and add/norm again
        x_norm2 = self.norm2(x)
        ffn_out = self.ffn(x_norm2)
        x = x + ffn_out

        return x


In [41]:
# "encoder" (not encoderblock), will stack the tformerblock
class Encoder(nn.Module):
    def __init__(self, input_dim, d_model, num_heads, d_ff, num_layers=2):
        super().__init__()

        # must make sure input is correct dimension (? double check)
        self.input_proj = nn.Linear(input_dim, d_model)

        # setup block layers
        self.layers = nn.ModuleList([
            TransformerBlock(d_model, num_heads, d_ff)
            for _ in range(num_layers)
        ])

    def forward(self, x):
        x = self.input_proj(x)

        for i, layer in enumerate(self.layers):
            x = layer(x)

        return x

In [47]:
# "main" to test just the model code
x = torch.randn(batch_size, seq_len, input_dim)
encoder = Encoder(input_dim, d_model, num_heads, d_ff, num_layers=2)

output = encoder(x)
print(output.shape)
print(output)
print(output.grad)

torch.Size([2, 4, 8])
tensor([[[ 0.6514, -1.1451, -0.6806, -1.2014,  0.1812,  1.6664, -2.6184,
           0.7552],
         [-0.6710, -1.4770,  0.1584, -0.7289, -0.1058,  1.2851, -0.3658,
           0.8349],
         [-0.5325, -0.3263, -0.6118, -0.2680,  1.5829,  0.2944, -1.1985,
          -0.7723],
         [-0.1372, -0.8505,  0.3939, -0.6812,  0.5379,  0.4848, -0.5404,
           0.0939]],

        [[-0.1782, -0.9927, -0.4383, -0.3518,  0.3448,  1.1385, -0.3957,
           0.7862],
         [-0.1746, -1.0193,  0.3260, -0.8109,  0.9293,  0.1760, -0.4882,
          -0.4234],
         [ 0.4474, -1.5515, -0.4870, -0.6148,  0.8368,  0.2902, -0.2324,
           0.1455],
         [ 0.0474, -0.3549,  0.2178, -0.2285,  0.8008,  0.5741, -1.9748,
           0.2921]]], grad_fn=<AddBackward0>)
None


/tmp/ipython-input-945291916.py:8: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  print(output.grad)
